In [2]:
%run 'C:\dev\prod\metrikaHelper.py'
import pandas as pd
import psycopg2
import numpy as np
def getVipProjectByDomain(domain):
    conn = psycopg2.connect("dbname='CRM' user='read_only' host='192.168.10.10' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT p.projectid, p.domain, p2.login, p2.password FROM project AS p INNER JOIN ProjectMetrikaData AS p2 ON p.UniqueId = p2.ProjectUniqueId WHERE domain=%s ORDER BY p.projectid", [domain])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getVipProjectByRow(row))
    return result
def getVipProjectByRow(row):
    return {'projectid': row[0], 'domain': row[1], 'metrikalogin': row[2], 'metrikapassword': row[3]}
def getCPATargetsByProjectID(projectID):
    conn = psycopg2.connect("dbname='Cubo' user='read_only' host='192.168.10.8' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT targetstatus, targetid, targetname FROM metricstarget WHERE projectid=%s ORDER BY id DESC", [projectID])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getCPATargetByRow(row))
    return result
def getCPATargetByRow(row):
    return {'targetstatus': row[0], 'targetid': row[1], 'targetname': row[2]}
project = u'syntone.ru'
VipProjectInfo = getVipProjectByDomain(project)
print VipProjectInfo
metrikaCredentials = getMetrikaCredentialsByLPD({'yandexLogin':VipProjectInfo[0]['metrikalogin'],
                                                 'yandexPassword':VipProjectInfo[0]['metrikapassword'],
                                                 'domain':VipProjectInfo[0]['domain']})

print metrikaCredentials

[{'metrikalogin': 'cubo.client2@yandex.ru', 'projectid': 2434227L, 'domain': 'syntone.ru', 'metrikapassword': 'idev2015'}, {'metrikalogin': '', 'projectid': 2444069L, 'domain': 'syntone.ru', 'metrikapassword': ''}, {'metrikalogin': None, 'projectid': None, 'domain': 'syntone.ru', 'metrikapassword': None}]
{'Token': u'00687c8af3aa4b4bbddea008faae5bfa', 'counterID': [36148300, 12271228]}


In [7]:
goalsMetrics = ','.join(['ym:s:goal'+str(target['targetid'])+'visits' for target in getCPATargetsByProjectID(VipProjectInfo[0]['projectid'])])
otherMetrics = 'ym:s:visits'
metrics = ','.join([otherMetrics,goalsMetrics])
dimensions='ym:s:UTMSource,ym:s:UTMMedium,ym:s:UTMCampaign,ym:s:UTMContent,ym:s:UTMTerm'
date1 = '2016-03-19'
date2 = '2016-04-04'
token = metrikaCredentials['Token']
counterId = metrikaCredentials['counterID'][1]
params = '&'.join(['dimensions={0}'.format(dimensions),
                   'metrics={0}'.format(metrics),
                   'date1={0}'.format(date1),
                  'date2={0}'.format(date2),
                  'ids={0}'.format(counterId),
                  'oauth_token={0}'.format(token),
                  'limit=100000',
                  'offset=1',
                  "filters=ym:s:UTMSource=='cubo'"])
url = 'https://beta.api-metrika.yandex.ru/stat/v1/data.csv?'+params
print url

https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=ym:s:UTMSource,ym:s:UTMMedium,ym:s:UTMCampaign,ym:s:UTMContent,ym:s:UTMTerm&metrics=ym:s:visits,ym:s:goal17200540visits,ym:s:goal17200530visits,ym:s:goal17200520visits,ym:s:goal17200535visits,ym:s:goal17200525visits&date1=2016-03-19&date2=2016-04-04&ids=12271228&oauth_token=00687c8af3aa4b4bbddea008faae5bfa&limit=100000&offset=1&filters=ym:s:UTMSource=='cubo'
